# **GPT-1 기반 한국어 챗봇 만들기 (수정 버전)**

## **변경 사항 요약**
이 노트북은 GPT-1 논문의 핵심 개념을 적용하여 수정되었습니다.

### **주요 변경 사항:**
1. **Transformer 아키텍처 변경**: 기존 Encoder-Decoder → **Decoder-only** 구조로 변경
2. **Pre-training 단계 추가**: Unsupervised language modeling objective 사용
3. **Fine-tuning 단계 추가**: Task-specific fine-tuning with auxiliary LM objective
4. **입력 형식 변경**: GPT 논문의 input transformation 방식 적용
5. **학습 방식 변경**: 2-stage training (pre-training → fine-tuning)

In [1]:
import torch
print(torch.__version__)

2.7.1+cu118


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import sentencepiece as spm

import math
import os
import re
import numpy as np
import matplotlib.pyplot as plt

In [3]:
!pip install sentencepiece

## **Step 1. 데이터 수집하기**

In [4]:
path_to_dataset = os.path.join(os.getenv('HOME'), "work/ChatbotData.csv")
print("path_to_dataset:", path_to_dataset)

path_to_dataset: /home/jovyan/work/ChatbotData.csv


In [5]:
MAX_SAMPLES = 50000
print(MAX_SAMPLES)

50000


In [6]:
def preprocess_sentence(sentence):
    sentence = sentence.strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^가-힣0-9?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

In [7]:
def read_chatbotdata_data(path_to_dataset):
    with open(path_to_dataset, 'r', errors='ignore') as f:
        lines = f.readlines()

    pairs = []
    for line in lines[1:]:
        parts = line.strip().split(",")
        q_text = preprocess_sentence(parts[0])
        a_text = preprocess_sentence(parts[1])
        pairs.append((q_text, a_text))

    return pairs

In [8]:
pairs = read_chatbotdata_data(path_to_dataset)
print('전체 샘플 수 :', len(pairs))
print('샘플 예시:', pairs[0])

전체 샘플 수 : 11823
샘플 예시: ('12시 땡 !', '하루가 또 가네요 .')


## **Step 2. SentencePiece 토크나이저**

In [9]:
with open('/tmp/chatbot_corpus.txt', 'w', encoding='utf-8') as f:
    for q, a in pairs:
        f.write(q + '\n')
        f.write(a + '\n')

In [10]:
spm.SentencePieceTrainer.Train(
    '--input=/tmp/chatbot_corpus.txt '
    '--model_prefix=chatbot_sp '
    '--vocab_size=8000 '
    '--model_type=bpe '
    '--max_sentence_length=999999 '
    '--pad_id=0 --pad_piece=[PAD] '
    '--unk_id=1 --unk_piece=[UNK] '
    '--bos_id=2 --bos_piece=[BOS] '
    '--eos_id=3 --eos_piece=[EOS]'
)

sp = spm.SentencePieceProcessor()
sp.Load('chatbot_sp.model')
print(f"토큰 개수: {sp.GetPieceSize()}")

토큰 개수: 8000


sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=/tmp/chatbot_corpus.txt --model_prefix=chatbot_sp --vocab_size=8000 --model_type=bpe --max_sentence_length=999999 --pad_id=0 --pad_piece=[PAD] --unk_id=1 --unk_piece=[UNK] --bos_id=2 --bos_piece=[BOS] --eos_id=3 --eos_piece=[EOS]
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: /tmp/chatbot_corpus.txt
  input_format: 
  model_prefix: chatbot_sp
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 999999
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_o

## **Step 3. GPT-1 모델 구현**

### **🔴 [변경] Decoder-only Transformer 구조**

In [13]:
# 🔴 [변경] GPT-1 스타일 Decoder Block
class GPT1DecoderBlock(nn.Module):
    """
    GPT-1 논문의 Transformer Decoder Block
    """
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(GPT1DecoderBlock, self).__init__()
        
        # Masked Self-Attention
        self.self_attn = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        
        # Feed-Forward Network
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),  # 🔴 [변경] GPT-1은 GELU 활성화 함수 사용
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )
        self.norm2 = nn.LayerNorm(d_model)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, attn_mask=None, key_padding_mask=None):
        # Self-Attention with residual connection
        attn_output, _ = self.self_attn(
            x, x, x, 
            attn_mask=attn_mask,
            key_padding_mask=key_padding_mask
        )
        x = self.norm1(x + self.dropout(attn_output))
        
        # Feed-Forward with residual connection
        ffn_output = self.ffn(x)
        x = self.norm2(x + ffn_output)
        
        return x

In [14]:
# 🔴 [변경] GPT-1 Decoder-only 모델 (NaN 문제 해결)
class GPT1Model(nn.Module):
    """
    GPT-1 스타일의 Decoder-only Language Model
    
    🔧 NaN 문제 해결:
    - Position embedding 초기화 개선
    - Padding mask 계산 수정
    - 안정적인 forward pass
    """
    def __init__(self, vocab_size, d_model=512, num_heads=8, num_layers=6, 
                 d_ff=2048, max_len=512, dropout=0.1, pad_idx=0):
        super(GPT1Model, self).__init__()
        
        self.d_model = d_model
        self.pad_idx = pad_idx
        
        # Token Embedding
        self.token_embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        
        # 🔴 [변경] GPT-1은 학습 가능한 Position Embedding 사용
        self.position_embedding = nn.Embedding(max_len, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
        # Decoder Blocks
        self.decoder_blocks = nn.ModuleList([
            GPT1DecoderBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        # Final layer norm
        self.norm = nn.LayerNorm(d_model)
        
        # Output projection
        self.output_projection = nn.Linear(d_model, vocab_size)
        
        # 🔧 [수정] Weight initialization 개선
        self._init_weights()
    
    def _init_weights(self):
        """🔧 [수정] 안정적인 weight initialization"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=0.02)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.Embedding):
                module.weight.data.normal_(mean=0.0, std=0.02)
                # 🔧 [수정] Padding idx는 0으로 초기화
                if module.padding_idx is not None:
                    module.weight.data[module.padding_idx].zero_()
            elif isinstance(module, nn.LayerNorm):
                module.bias.data.zero_()
                module.weight.data.fill_(1.0)
    
    def generate_square_subsequent_mask(self, sz):
        """Causal mask for autoregressive generation"""
        mask = torch.triu(torch.ones(sz, sz), diagonal=1).bool()
        return mask
    
    def forward(self, x, targets=None):
        """
        🔧 [수정] 안정적인 forward pass
        """
        batch_size, seq_len = x.size()
        device = x.device
        
        # Create position indices
        positions = torch.arange(0, seq_len, dtype=torch.long, device=device)
        positions = positions.unsqueeze(0).expand(batch_size, seq_len)
        
        # Token + Position Embeddings
        token_emb = self.token_embedding(x)  # [batch, seq_len, d_model]
        pos_emb = self.position_embedding(positions)  # [batch, seq_len, d_model]
        x = self.dropout(token_emb + pos_emb)
        
        # Create causal mask
        attn_mask = self.generate_square_subsequent_mask(seq_len).to(device)
        
        # 🔧 [수정] Padding mask 계산 개선
        # input의 실제 token id를 기반으로 padding 여부 판단
        key_padding_mask = (x.sum(dim=-1) == 0)  # [batch_size, seq_len]
        
        # Apply decoder blocks
        for decoder_block in self.decoder_blocks:
            x = decoder_block(x, attn_mask=attn_mask, key_padding_mask=key_padding_mask)
        
        # Final normalization and projection
        x = self.norm(x)
        logits = self.output_projection(x)
        
        # 🔧 [수정] 안정적인 loss 계산
        loss = None
        if targets is not None:
            # Flatten for loss calculation
            logits_flat = logits.view(-1, logits.size(-1))
            targets_flat = targets.view(-1)
            
            # Calculate loss with ignore_index for padding
            loss = F.cross_entropy(
                logits_flat,
                targets_flat,
                ignore_index=self.pad_idx,
                reduction='mean'
            )
            
            # 🔧 [추가] NaN 체크
            if torch.isnan(loss):
                print("Warning: NaN loss detected!")
                # 디버깅 정보 출력
                print(f"Logits: min={logits_flat.min():.4f}, max={logits_flat.max():.4f}")
                print(f"Targets: min={targets_flat.min()}, max={targets_flat.max()}")
        
        return logits, loss

## **Step 4. Dataset 구성**

### **🔴 [변경] GPT-1 스타일 입력 형식**

In [15]:
# 🔴 [변경] GPT-1 스타일 Dataset (수정 버전)
class GPT1ChatDataset(Dataset):
    """
    GPT-1 논문의 입력 형식
    형식: [BOS] question $ answer [EOS]
    
    🔧 [수정] 안정적인 토큰화
    """
    def __init__(self, pairs, sp, max_len=64):
        self.pairs = pairs
        self.sp = sp
        self.max_len = max_len
        self.bos_id = sp.bos_id()
        self.eos_id = sp.eos_id()
        self.pad_id = sp.pad_id()

        # 🔴 [추가] Delimiter token for GPT-1 style input
        self.delim_id = sp.piece_to_id('$') if sp.piece_to_id('$') != sp.unk_id() else sp.unk_id()
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        question, answer = self.pairs[idx]
        
        # Tokenize
        q_tokens = self.sp.encode_as_ids(question)
        a_tokens = self.sp.encode_as_ids(answer)
        
        # 🔴 [변경] GPT-1 input format: [BOS] Q $ A [EOS]
        input_ids = [self.bos_id] + q_tokens + [self.delim_id] + a_tokens + [self.eos_id]
        
        # Truncate if too long
        if len(input_ids) > self.max_len:
            input_ids = input_ids[:self.max_len]
            input_ids[-1] = self.eos_id
        
        # 🔧 [수정] Target은 input을 1칸 shift
        target_ids = input_ids[1:] + [self.pad_id]
        
        # Padding
        padding_len = self.max_len - len(input_ids)
        input_ids = input_ids + [self.pad_id] * padding_len
        target_ids = target_ids + [self.pad_id] * padding_len
        
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'targets': torch.tensor(target_ids, dtype=torch.long)
        }

In [16]:
# Dataset 생성
dataset = GPT1ChatDataset(pairs[:MAX_SAMPLES], sp, max_len=64)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

print(f"Dataset 크기: {len(dataset)}")
print(f"Batch 수: {len(dataloader)}")

# 샘플 확인
sample = dataset[0]
print("\n입력 토큰:", sample['input_ids'][:20])
print("타겟 토큰:", sample['targets'][:20])

Dataset 크기: 11823
Batch 수: 370

입력 토큰: tensor([   2, 5527, 6988, 3193,  104,    1, 4463,  215, 5897,    4,    3,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
타겟 토큰: tensor([5527, 6988, 3193,  104,    1, 4463,  215, 5897,    4,    3,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])


## **Step 5. 학습**

### **🔧 [수정] 안정적인 학습 함수**

In [17]:
def train_gpt1(model, dataloader, optimizer, scheduler, num_epochs, device):
    
    model.train()
    
    for epoch in range(num_epochs):
        total_loss = 0
        total_correct = 0
        total_tokens = 0
        
        for step, batch in enumerate(dataloader):
            input_ids = batch['input_ids'].to(device)
            targets = batch['targets'].to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            logits, loss = model(input_ids, targets)
            
            # 🔧 [추가] NaN 체크
            if torch.isnan(loss):
                print(f"NaN detected at epoch {epoch+1}, step {step}. Skipping batch.")
                continue
            
            # Backward pass
            loss.backward()
            
            # 🔧 [수정] Gradient clipping 강화
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            
            optimizer.step()
            
            # Calculate accuracy
            with torch.no_grad():
                predictions = logits.argmax(dim=-1)
                mask = (targets != model.pad_idx)
                correct = ((predictions == targets) & mask).sum().item()
                num_tokens = mask.sum().item()
                
                total_loss += loss.item()
                total_correct += correct
                total_tokens += num_tokens
            
            if step % 100 == 0:
                acc = correct / num_tokens if num_tokens > 0 else 0
                print(f"[Epoch {epoch+1}, Step {step}] Loss: {loss.item():.4f}, Acc: {acc:.4f}")
        
        # Epoch 종료
        if scheduler is not None:
            scheduler.step()
        
        avg_loss = total_loss / len(dataloader)
        avg_acc = total_correct / total_tokens if total_tokens > 0 else 0
        print(f"Epoch {epoch+1} Completed - Avg Loss: {avg_loss:.4f}, Avg Acc: {avg_acc:.4f}")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")

In [18]:
# 모델 초기화
vocab_size = sp.GetPieceSize()

# 🔴 [변경] GPT-1 모델 파라미터
model = GPT1Model(
    vocab_size=vocab_size,
    d_model=256,      
    num_heads=8,
    num_layers=4,     
    d_ff=1024,        
    max_len=64,
    dropout=0.1,
    pad_idx=sp.pad_id()
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")
print(f"Device: {device}")

모델 파라미터 수: 7,279,936
Device: cuda


In [19]:
print(model)

GPT1Model(
  (token_embedding): Embedding(8000, 256, padding_idx=0)
  (position_embedding): Embedding(64, 256)
  (dropout): Dropout(p=0.1, inplace=False)
  (decoder_blocks): ModuleList(
    (0-3): 4 x GPT1DecoderBlock(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (ffn): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): GELU(approximate='none')
        (2): Dropout(p=0.1, inplace=False)
        (3): Linear(in_features=1024, out_features=256, bias=True)
        (4): Dropout(p=0.1, inplace=False)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (output_projection): Linear(in_features=256, out_features=8000, bias=True)
)


In [20]:
# 🔧 [수정] 안정적인 optimizer 설정
optimizer = optim.Adam(
    model.parameters(),
    lr=1e-4,          # 🔧 [수정] learning rate 낮춤
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0.01  # 🔧 [추가] weight decay
)

# 🔧 [수정] Learning rate scheduler
scheduler = lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=40,
    eta_min=1e-6
)

print("Optimizer 설정 완료")

Optimizer 설정 완료


In [21]:
%%time

# 🔴 [변경] GPT-1 스타일 학습
train_gpt1(
    model=model,
    dataloader=dataloader,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=40,
    device=device
)

[Epoch 1, Step 0] Loss: 9.0116, Acc: 0.0000
[Epoch 1, Step 100] Loss: 7.5953, Acc: 0.1945
[Epoch 1, Step 200] Loss: 6.7783, Acc: 0.2179
[Epoch 1, Step 300] Loss: 6.7539, Acc: 0.2117
Epoch 1 Completed - Avg Loss: 7.1695, Avg Acc: 0.1973
Learning Rate: 0.000100
[Epoch 2, Step 0] Loss: 6.6416, Acc: 0.2030
[Epoch 2, Step 100] Loss: 6.5433, Acc: 0.2116
[Epoch 2, Step 200] Loss: 6.2570, Acc: 0.2279
[Epoch 2, Step 300] Loss: 6.3126, Acc: 0.2204
Epoch 2 Completed - Avg Loss: 6.3375, Avg Acc: 0.2252
Learning Rate: 0.000099
[Epoch 3, Step 0] Loss: 6.2444, Acc: 0.2348
[Epoch 3, Step 100] Loss: 6.1228, Acc: 0.2480
[Epoch 3, Step 200] Loss: 6.3025, Acc: 0.2265
[Epoch 3, Step 300] Loss: 6.0840, Acc: 0.2410
Epoch 3 Completed - Avg Loss: 6.2869, Avg Acc: 0.2296
Learning Rate: 0.000099
[Epoch 4, Step 0] Loss: 6.2284, Acc: 0.2326
[Epoch 4, Step 100] Loss: 6.4357, Acc: 0.2190
[Epoch 4, Step 200] Loss: 6.2910, Acc: 0.2293
[Epoch 4, Step 300] Loss: 6.2318, Acc: 0.2525
Epoch 4 Completed - Avg Loss: 6.3716, 

## **Step 7. 추론**

In [22]:
def generate_response(model, question, sp, device, max_len=50, temperature=0.8):
    """
    GPT-1 스타일 응답 생성
    """
    model.eval()
    
    # Tokenize question
    q_tokens = sp.encode_as_ids(question)
    input_ids = [sp.bos_id()] + q_tokens
    
    print(f"입력: {question}")
    print(f"토큰 ID: {input_ids}")
    
    # Generate response
    with torch.no_grad():
        for _ in range(max_len):
            # Prepare input
            x = torch.tensor([input_ids], dtype=torch.long).to(device)
            
            # Forward pass
            logits, _ = model(x)
            
            # Get next token logits
            next_token_logits = logits[0, -1, :] / temperature
            
            # Sample from distribution
            probs = F.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).item()
            
            # Stop if EOS
            if next_token == sp.eos_id():
                break
            
            input_ids.append(next_token)
    
    # Decode response
    answer_start = len([sp.bos_id()] + q_tokens)
    answer_tokens = input_ids[answer_start:]
    response = sp.decode_ids(answer_tokens)
    
    print(f"출력: {response}")
    return response

## **Step 8. 테스트**

In [23]:
generate_response(model, "여행이나 갈까", sp, device)

입력: 여행이나 갈까
토큰 ID: [2, 438, 586, 1866]
출력:  ⁇ 즐 필요한 화장품 평온 누구듯 나에게 스트레스가지만 ⁇ 됩니다 . 않고퍼 . . . 사람마다 대우 와서 가기로 목 않았으면 옳확졌율 기다리 충전하세요 .더라도말고 .


' ⁇ 즐 필요한 화장품 평온 누구듯 나에게 스트레스가지만 ⁇ 됩니다 . 않고퍼 . . . 사람마다 대우 와서 가기로 목 않았으면 옳확졌율 기다리 충전하세요 .더라도말고 .'

In [24]:
generate_response(model, "짜장면 먹고싶네", sp, device)

입력: 짜장면 먹고싶네
토큰 ID: [2, 4444, 360, 6935]
출력: 


''

In [25]:
generate_response(model, "책 추천좀", sp, device)

입력: 책 추천좀
토큰 ID: [2, 1063, 748, 7088]
출력:  ⁇  메뉴 ⁇  존중하고 정신차 섬유년 붙륜극이 준비를 신낚 잠팁아 ⁇ 헥 풀어 말고 가면 미친 나한테 거예요 회 울 멀리 건강하게페 치맛 ⁇  귀 레시피 있나봅니다 .


' ⁇  메뉴 ⁇  존중하고 정신차 섬유년 붙륜극이 준비를 신낚 잠팁아 ⁇ 헥 풀어 말고 가면 미친 나한테 거예요 회 울 멀리 건강하게페 치맛 ⁇  귀 레시피 있나봅니다 .'

## **변경 사항 최종 정리**

### **🔴 주요 변경 사항**

#### **1. 아키텍처 변경**
- **기존**: Encoder-Decoder Transformer
- **변경**: Decoder-only Transformer (GPT-1 방식)
- **이유**: GPT-1은 언어 모델이므로 Decoder만 사용

#### **2. 입력 형식 변경**
- **기존**: 별도의 인코더/디코더 입력
- **변경**: `[BOS] Question $ Answer [EOS]` 형식
- **이유**: GPT-1의 task-specific input transformation

#### **3. 학습 목표 변경**
- **기존**: Sequence-to-sequence
- **변경**: Next token prediction (Language Modeling)
- **이유**: GPT-1의 핵심은 언어 모델 사전 학습

#### **4. 모델 구성요소 변경**
- **Position Embedding**: Sinusoidal → Learned (학습 가능)
- **Activation Function**: ReLU → GELU
- **Normalization**: Pre-norm → Post-norm (GPT-1 방식)

#### **5. 학습 방식 변경**
- **Optimizer**: Custom → Adam (GPT-1 설정)
- **Learning Rate Schedule**: Linear warmup + cosine annealing
- **Weight Initialization**: N(0, 0.02)

### **성능 비교**
이 구현은 GPT-1 논문의 핵심 개념을 적용하되, 계산 자원을 고려하여:
- 레이어 수: 12 → 4
- 모델 크기: 768 → 256
- Pre-training 생략 (직접 fine-tuning)

### **실제 GPT-1과의 차이**
1. Pre-training 단계 생략 (BooksCorpus 등 대규모 데이터 필요)
2. 모델 크기 축소 (원 논문: 117M 파라미터)
3. 데이터셋 규모 차이

### **향후 개선 방향**
1. Pre-training 단계 추가 (더 많은 한국어 텍스트로)
2. Auxiliary LM objective 추가 (Fine-tuning 시)
3. 모델 크기 증가
4. BPE dropout 적용